## Cat reconstructed by different polynomial degree

resolution of 32 32 32 

plot 1: k=2 wendlandradius=16

plot 2: k=1 wendlandradius=16

plot 3: k=0 wendlandradius=16

plot 4: k=0 wendlandradius=8

For a given Wendland radius, increasing the degreek) introduces more artifacts in the reconstructio

Meanwhile, with a lower degree, it's possible to decrease the Wendland radius without compromising reconstruction quality, a strategy that doesn't work as well with higher degrees. 

This finding suggests that careful selection of both degree and Wendland radius is crucial for optimizing reconstruction resultsn.

In [43]:
_,_,_,iframe4=MyWholeConstructionInOneFunc(filelocation="data/cat.off", normalizationmethod=1, spatial=False, plotconstrain=False,plotinout=False, 
                             n=[32,32,32],point_size=5,plotgrid=False,coefficientnumber=10,plotreconstructed=True,gridlength=16,k=2,
                             wendlandRadius=16,adjindices=None)


Plot saved to file ./htmls/plot_24.html.


In [44]:
IFrame(src=iframe4, width=700, height=600)

In [45]:
_,_,_,iframe4=MyWholeConstructionInOneFunc(filelocation="data/cat.off", normalizationmethod=1, spatial=False, plotconstrain=False,plotinout=False, 
                             n=[32,32,32],point_size=5,plotgrid=False,coefficientnumber=4,plotreconstructed=True,gridlength=16,k=1,
                             wendlandRadius=16,adjindices=None)


Plot saved to file ./htmls/plot_25.html.


In [46]:
IFrame(src=iframe4, width=700, height=600)

In [47]:
_,_,_,iframe4=MyWholeConstructionInOneFunc(filelocation="data/cat.off", normalizationmethod=1, spatial=False, plotconstrain=False,plotinout=False, 
                             n=[32,32,32],point_size=5,plotgrid=False,coefficientnumber=1,plotreconstructed=True,gridlength=16,k=0,
                             wendlandRadius=16,adjindices=None)


Plot saved to file ./htmls/plot_26.html.


In [48]:
IFrame(src=iframe4, width=700, height=600)

In [49]:
_,_,_,iframe4=MyWholeConstructionInOneFunc(filelocation="data/cat.off", normalizationmethod=1, spatial=False, plotconstrain=False,plotinout=False, 
                             n=[32,32,32],point_size=5,plotgrid=False,coefficientnumber=1,plotreconstructed=True,gridlength=9,k=0,
                             wendlandRadius=9,adjindices=None)


Plot saved to file ./htmls/plot_27.html.


In [50]:
IFrame(src=iframe4, width=700, height=600)

# Optional Task 1:

In [51]:
def calculatefx_trueconstraint(x, npi,npiplus, npiminus, ni, bbox_min, coefficientnumber, bbox_max, gridlength, spatial,
                wendlandRadius, k,adjindices):
    funcargs = np.zeros((x.shape[0], coefficientnumber))  # 1,x,y,z,x^2,xy,xz,y^2,yz,z^2
    points = np.concatenate((npi, npiplus, npiminus))
    if spatial == True:
        gridcells, gridcellnumbers = constructspatialgrid(points, bbox_min, bbox_max, gridlength)
    fx = np.zeros((x.shape[0], 1))
    for (i, xi) in enumerate(x):
        if spatial == False:
            adjlist = np.array(closest_points(xi, points, wendlandRadius))
        else:
            adjlist = np.array(
                closest_points_spatial(xi, points, bbox_min, gridcells, gridcellnumbers, gridlength, wendlandRadius,adjindices))
        a = np.zeros((adjlist.size, coefficientnumber))
        b = np.zeros((adjlist.size, 1))
        w = np.zeros((adjlist.size, adjlist.size))  # diagonal matrix
        if (len(adjlist) < coefficientnumber):
            fx[i] = 10000
            continue
        for (j, adj) in enumerate(adjlist):
            adj_position = points[adj]
            originalpiindex = adj % (npi.shape[0])
            originalpi_position = points[originalpiindex]
            eps = (adj_position - originalpi_position)[0] / ni[originalpiindex, 0]
            Sx=eps+ ((xi-adj_position).T @ ni[originalpiindex])
            a[j, :] = functioninvector(adj_position[0], adj_position[1], adj_position[2], k)
            b[j] = Sx  # what would be range?
            w[j, j] = wendlandweight(adj_position, xi, wendlandRadius)
        funcargs[i] = np.linalg.solve(a.T @ w @ a, a.T @ w @ b).T
        fx[i] = (functioninvector(xi[0], xi[1], xi[2], k) @ funcargs[i]).T
    return fx, funcargs

def MyWholeConstructionInOneFunc_trueconstraint(filelocation, normalizationmethod, spatial, plotconstrain, plotinout, n,
                                 point_size, plotgrid, coefficientnumber, plotreconstructed, gridlength, k,
                                 wendlandRadius,adjindices,tohtml=True,trueconstraint=True):  # functioninvector
    pi, v= readMesh(filelocation)
    npi, bbox_min, bbox_max, gridcells, gridcellnumbers, ni = normalizeMesh(v,normalizationmethod, pi, spatial, gridlength)
    npiplus, npiminus = constructConstraint(npi, ni, spatial, bbox_min, gridcells, gridcellnumbers, gridlength,adjindices)
    if plotconstrain == True:
        print("The following plot is the required output for Setting up the Constraints section")
        p = mp.plot(npi, c=np.ones_like(npi) * np.array([0, 0, 1]), shading={"point_size": point_size})
        p.add_points(npiplus, c=np.ones_like(npi) * np.array([1, 0, 0]), shading={"point_size": point_size})
        p.add_points(npiminus, c=np.ones_like(npi) * np.array([0, 1, 0]), shading={"point_size": point_size})
        if tohtml==True:
            htmlit(p)
    x, T = tet_grid([n[0], n[1], n[2]], bbox_min, bbox_max)
    if plotgrid == True:
        print("The following plot is the required output for Using a non-axis-aligned grid section")
        p = mp.plot(npi, v)
        p.add_points(x, shading={"point_size": point_size})
        if tohtml==True:
            htmlit(p)
    fx, funcargs = calculatefx_trueconstraint(x, npi,npiplus, npiminus, ni, bbox_min, coefficientnumber, bbox_max, gridlength,
                               spatial, wendlandRadius, k,adjindices)
    ind = virtualizeinout(fx)
    if plotinout == True:
        print("The following plot is the required output for \"Use MLS interpolation to extend to function f\" section")
        p = mp.plot(x, c=ind, shading={"point_size": point_size, "width": 800, "height": 800})
        if tohtml==True:
            htmlit(p)
    sv, filteredf = marching(x, T, fx)
    if plotreconstructed == True:
        p=mp.plot(sv, filteredf, shading={"wireframe": True})
        if tohtml==True:
            iframe4=htmlit(p)
    return fx,iframe4

def MyWholeConstructionInOneFunc(filelocation, normalizationmethod, spatial, plotconstrain, plotinout, n,
                                 point_size, plotgrid, coefficientnumber, plotreconstructed, gridlength, k,
                                 wendlandRadius,adjindices,tohtml=True):  # functioninvector
    iframe4=None
    pi, v= readMesh(filelocation)
    npi, bbox_min, bbox_max, gridcells, gridcellnumbers, ni = normalizeMesh(v,normalizationmethod, pi, spatial, gridlength)
    npiplus, npiminus = constructConstraint(npi, ni, spatial, bbox_min, gridcells, gridcellnumbers, gridlength,adjindices)
    if plotconstrain == True:
        print("The following plot is the required output for Setting up the Constraints section")
        p = mp.plot(npi, c=np.ones_like(npi) * np.array([0, 0, 1]), shading={"point_size": point_size})
        p.add_points(npiplus, c=np.ones_like(npi) * np.array([1, 0, 0]), shading={"point_size": point_size})
        p.add_points(npiminus, c=np.ones_like(npi) * np.array([0, 1, 0]), shading={"point_size": point_size})
        if tohtml==True:
            htmlit(p)
    x, T = tet_grid([n[0], n[1], n[2]], bbox_min, bbox_max)
    if plotgrid == True:
        print("The following plot is the required output for Using a non-axis-aligned grid section")
        p = mp.plot(npi, v)
        p.add_points(x, shading={"point_size": point_size})
        if tohtml==True:
            htmlit(p)
    fx, funcargs = calculatefx(x, npi, npiplus, npiminus, ni, bbox_min, coefficientnumber, bbox_max, gridlength,
                               spatial, wendlandRadius, k,adjindices)
    ind = virtualizeinout(fx)
    if plotinout == True:
        print("The following plot is the required output for \"Use MLS interpolation to extend to function f\" section")
        p = mp.plot(x, c=ind, shading={"point_size": point_size, "width": 800, "height": 800})
        if tohtml==True:
            htmlit(p)
    sv, filteredf = marching(x, T, fx)
    if plotreconstructed == True:
        p=mp.plot(sv, filteredf, shading={"wireframe": True})
        if tohtml==True:
            iframe4=htmlit(p)
    return fx,iframe4

resolution of 32 32 32

wendlandRadius=16

k=0

Plot 1: point constraints (original one)

Plot 2: true normal constraints (modified one)

In this comparison, the two plots look very similar, showing that both ways of setting constraints work almost the same for reconstructing the object. We added a bit of code to check the differences more closely. This check found that only 40 points disagree on if they are part of the cat or not. This tells us that both methods are quite good at getting the shape of the cat right, with just a few small differences.

In [52]:
fx1,iframe4=MyWholeConstructionInOneFunc(filelocation="data/cat.off", normalizationmethod=1, spatial=False, plotconstrain=False,plotinout=False, 
                             n=[32,32,32],point_size=5,plotgrid=False,coefficientnumber=1,plotreconstructed=True,gridlength=16,k=0,
                             wendlandRadius=16,adjindices=None)
IFrame(src=iframe4, width=700, height=600)

Plot saved to file ./htmls/plot_28.html.


In [53]:
fx2,iframe4=MyWholeConstructionInOneFunc_trueconstraint(filelocation="data/cat.off", normalizationmethod=1, spatial=False, plotconstrain=False,plotinout=False, 
                             n=[32,32,32],point_size=5,plotgrid=False,coefficientnumber=1,plotreconstructed=True,gridlength=16,k=0,
                             wendlandRadius=16,adjindices=None)
IFrame(src=iframe4, width=700, height=600)

Plot saved to file ./htmls/plot_29.html.


In [54]:
differentfx=0
for i, x in enumerate(fx1):
    if(fx1[i]*fx2[i]<0):
        differentfx+=1
print(str(differentfx))

40


# Optional Task 2

There is a pymeshlab library, which is easy to implement in ipynb. 

Please install it by `pip3 install pymeshlab`

Discussion for task 2:

Using screened Poisson for reconstructing Luigi showed better detail, like a jagged brim on his hat. It also used fewer triangles for flat surfaces with less detail. This method was much quicker, taking only 6.17 seconds, which is significantly faster than my previous algorithm.

For the cat model, screened Poisson was more accurate than my reconstruction technique. My method resulted in a solid base with artifacts and incorrectly marked many inside points as "outside," creating unwanted inner surfaces. Screened Poisson, however, created a clean surface of the cat without any artifacts.

In [55]:
%%time
import pymeshlab
import meshplot as mp

ms = pymeshlab.MeshSet()
ms.load_new_mesh('./data/luigi.off')
ms.generate_surface_reconstruction_screened_poisson()
m=ms.current_mesh()
v_matrix = m.vertex_matrix()
f_matrix = m.face_matrix()
p=mp.plot(v_matrix, f_matrix, shading={"point_size": 5,"wireframe":False})
html=htmlit(p)
IFrame(src=html, width=700, height=600)

Plot saved to file ./htmls/plot_30.html.
CPU times: total: 15 s
Wall time: 5.95 s


In [56]:
ms = pymeshlab.MeshSet()
ms.load_new_mesh('./data/cat.off')
ms.generate_surface_reconstruction_screened_poisson()
m=ms.current_mesh()
v_matrix = m.vertex_matrix()
v_matrix /= 10
f_matrix = m.face_matrix()
p=mp.plot(v_matrix, f_matrix, shading={"point_size": 5})
html=htmlit(p)
IFrame(src=html, width=700, height=600)

Plot saved to file ./htmls/plot_31.html.
